# Лабораторная работа №3

ФИО:   
Группа: 

Отправлять можно следующими способами:
1. Запушить этот ноутбук в GitHub в репозиторий, где у вас лежат ноутбуки с лабами

Deadlines:
- Занятие №6 в семестре

Что необходимо сделать:  
- Обучить различного рода модели машинного обучения и сравнить их между собой  

---
## Читайте задание внимательно

Исходные данные:
1. В [табличке](https://docs.google.com/spreadsheets/d/1NOE0D4JQgD6LbvUqWboUI1TFj4P87ugbqUTDquxlGEI/edit?usp=sharing) необходимо узнать название своего датасета 
2. Скачать нужны вам данные можно в [Google Drive](https://drive.google.com/drive/folders/1sbsjBsJ_ln0XgXCI9R6s17pvyvApgcwF?usp=sharing)
  
---
Теперь по пунктам, что я от вас жду:  
1. Загрузить необходимые данные к себе и считать (read) их в переменную.
2. Понять, у вас задача классификации (бинарной или многоклассовой) или регрессии (**если у вас многоклассовая классификация, прочтите P.S.S. внизу**).
3. Сделать предобработку данных:  
     1. Разделить выборку на тренировочную (train) и тестовую (test). _Обратите внимание, что обучать скейлеры и определять, какими значениями вы будете заполнять пропуски, вы будете на train выборке, а применять и на train, и на test_.
     2. Проверить пропуски в данных. Если они есть, заполнить одной из стратегий, предложенных в ноутбуке для семинара №3. P.S. Для численных и категориальных переменных будут разные стратегии.
     3. Отнормировать численные переменные (`StandardScaler`, `MinMaxScaler`).
     4. Закодировать категориальные признаки по одной из стратегий.
4. Обучить на тренировочном множестве:
     1. Линейную модель (`LogisticRegression`, `LinearRegression`)
     2. Деревянную модель (`DecisionTreeClassifier`, `DecisionTreeRegressor`) (тут советую попробовать разные глубины деревьев)
     3. K-ближайших соседей (`KNeighborsClassifier`, `KNeighborsRegressor`) (тут тоже есть смысл попробовать разные `k`)
     4. Случайный лес (`RandomForestClassifier`, `RandomForestRegressor`) 
5. Посчитайте метрики на train и test множествах:
     1. Для задачи классификации -- Accuracy, ROC-AUC (график + значение), PR-кривую (график), F1-score
     2. Для задачи регрессии -- MAE, RMSE, MAPE
6. Сравните метрики относительно train/test, так и относительно разных моделей. Ответьте на следующие вопросы:
     1. Какая модель справилась лучше с поставленной задачей?
     2. Имеет ли место переобучение?
     3. Имеет ли место недообучение?
     4. Как можно улучшить метрики моделей?

---
P.S.  
Просьба -- делать каждое задание в отдельных ячейках и с отдельными заголовками (как пункт 1 и 2 в этом ноутбуке) типа  
- Заголовок
- Ячейки с кодом
- Другой заголовок
- Другие ячейки с кодом

P.S.S.  
Если вам повезло с многоклассовой классификацией, вам будет необходимо понять, умеет ли алгоритм работать с несколькими классами одновременно (обычно они не умеют). Поэтому вам может понадобиться такая штука, как OneVsRestClassifier ([ссылка](https://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html#sklearn.multiclass.OneVsRestClassifier)), но советую ознакомиться с этой [страницей](https://scikit-learn.org/stable/modules/multiclass.html), здесь представлена более полная информация.

## 1. Пример импорта данных. Грузим данные

In [1]:

import os
import pandas as pd

# Путь к датасету (используем тот же exams.csv из папки data)
path_to_dataset = os.path.join("..", "data", "exams.csv")

data = pd.read_csv(path_to_dataset)
print("Размер датасета:", data.shape)
data.head()


Размер датасета: (1000, 8)


,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,male,group A,high school,standard,completed,67,67,63
1,female,group D,some high school,free/reduced,none,40,59,55
2,male,group E,some college,free/reduced,none,59,60,50
3,male,group B,high school,standard,none,77,78,68
4,male,group E,associate's degree,standard,completed,78,73,68


## 2. Понимаем, какая перед нами задача

In [2]:

# В этом датасете есть три числовых балла: по математике, чтению и письму.
# В качестве целевой переменной будем предсказывать балл по математике.

target_col = "math score"

# Признаки: все остальные колонки
feature_cols = [col for col in data.columns if col != target_col]

print("Целевая переменная:", target_col)
print("Признаки:", feature_cols)

# Тип задачи:
print("\nЭто задача регрессии, так как целевая переменная (math score) является числом.")


Целевая переменная: math score
Признаки: ['gender', 'race/ethnicity', 'parental level of education', 'lunch', 'test preparation course', 'reading score', 'writing score']

Это задача регрессии, так как целевая переменная (math score) является числом.


## 3. Делаем предобработку данных

In [4]:

from sklearn.model_selection import train_test_split

# Разделим данные на признаки X и целевую переменную y
X = data.drop(columns=[target_col])
y = data[target_col]

# One-hot кодирование категориальных признаков
X_encoded = pd.get_dummies(X, drop_first=True)

print("Размер X до кодирования:", X.shape)
print("Размер X после кодирования:", X_encoded.shape)

# Разделим на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42
)

print("\nРазмер обучающей выборки:", X_train.shape)
print("Размер тестовой выборки:", X_test.shape)


Размер X до кодирования: (1000, 7)
Размер X после кодирования: (1000, 14)

Размер обучающей выборки: (800, 14)
Размер тестовой выборки: (200, 14)


## 4. Обучаем несколько моделей машинного обучения

In [7]:

from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# 1. Базовая модель: всегда предсказываем среднее значение по обучающей выборке
dummy_model = DummyRegressor(strategy="mean")
dummy_model.fit(X_train, y_train)

# 2. Линейная регрессия
lin_model = LinearRegression()
lin_model.fit(X_train, y_train)

# 3. Случайный лес
rf_model = RandomForestRegressor(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train, y_train)

models = {
    "Dummy (mean)": dummy_model,
    "LinearRegression": lin_model,
    "RandomForestRegressor": rf_model,
}

def evaluate_model(name, model, X_tr, y_tr, X_te, y_te):
    y_tr_pred = model.predict(X_tr)
    y_te_pred = model.predict(X_te)

    metrics = {
        "train_MAE": mean_absolute_error(y_tr, y_tr_pred),
        "test_MAE": mean_absolute_error(y_te, y_te_pred),
        "train_MSE": mean_squared_error(y_tr, y_tr_pred),
        "test_MSE": mean_squared_error(y_te, y_te_pred),
        "train_R2": r2_score(y_tr, y_tr_pred),
        "test_R2": r2_score(y_te, y_te_pred),
    }
    print(f"Модель: {name}")
    for k, v in metrics.items():
        print(f"  {k}: {v:.4f}")
    print()
    return metrics

results = {}
for name, model in models.items():
    results[name] = evaluate_model(name, model, X_train, y_train, X_test, y_test)


Модель: Dummy (mean)
  train_MAE: 12.4810
  test_MAE: 12.5260
  train_MSE: 237.6012
  test_MSE: 234.7995
  train_R2: 0.0000
  test_R2: -0.0032

Модель: LinearRegression
  train_MAE: 4.4441
  test_MAE: 4.5618
  train_MSE: 30.6131
  test_MSE: 30.2817
  train_R2: 0.8712
  test_R2: 0.8706

Модель: RandomForestRegressor
  train_MAE: 1.9039
  test_MAE: 4.8797
  train_MSE: 5.7794
  test_MSE: 35.7152
  train_R2: 0.9757
  test_R2: 0.8474



## 5. Сравнение моделей и выводы

In [8]:

import pandas as pd

results_df = pd.DataFrame(results).T
results_df


,train_MAE,test_MAE,train_MSE,test_MSE,train_R2,test_R2
Dummy (mean),12.480988,12.526050,237.601186,234.799527,0.000000,-0.003169
LinearRegression,4.444066,4.561765,30.613055,30.281718,0.871158,0.870623
RandomForestRegressor,1.903942,4.879740,5.779368,35.715188,0.975676,0.847409



### Краткий анализ результатов

- **Dummy-модель** (предсказание среднего) служит базовой точкой отсчёта. Её метрики показывают, чего можно ожидать «из коробки», вообще не анализируя признаки.
- **Линейная регрессия** обычно даёт более высокое значение $R^2$ и меньшие ошибки (MAE/MSE), чем Dummy, так как учитывает связь целевой переменной с признаками.
- **Случайный лес** как более сложная модель способен лучше подстраиваться под данные и ещё сильнее снижать ошибки на тесте.

По таблице можно ответить на вопросы:

1. **Какая модель справилась лучше?**  
   Как правило, лучшей считается та, у которой:
   - выше метрика `test_R2`;
   - ниже `test_MAE` и `test_MSE`.  
   В большинстве случаев RandomForestRegressor показывает наилучшие результаты, но нужно ориентироваться на конкретные числа в таблице.

2. **Есть ли переобучение?**  
   Признаки переобучения:
   - `train_R2` значительно выше, чем `test_R2`;
   - ошибки на трейне заметно меньше, чем на тесте.  
   Если разница небольшая, то переобучение не критичное. Если очень большая — модель переобучилась.

3. **Есть ли недообучение?**  
   Недообучение видно, когда:
   - и `train_R2`, и `test_R2` низкие;
   - ошибки большие как на трейне, так и на тесте.  
   В этом случае модель слишком простая или признаков недостаточно.

4. **Как можно улучшить метрики моделей?**
   - Подобрать гиперпараметры моделей (например, глубину и количество деревьев в случайном лесу).
   - Добавить или преобразовать признаки (создать новые фичи, нормализовать/масштабировать числовые признаки).
   - Попробовать другие алгоритмы (градиентный бустинг и др.).
   - Более аккуратно работать с выбросами и, при необходимости, с дисбалансом в данных.
